In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyeit.mesh as mesh
from pyeit.eit.utils import eit_scan_lines

In [2]:
N_ELECTRODES = 16
IMAGE_PATH = '2cm_el1.jpg' # Change this to '1cm_el1.jpg' or '3cm_el1.jpg'
THRESHOLD_VAL = 150 # IMPORTANT: Tune this value (0-255) for best results!

In [3]:
def process_image_to_mask(image_path, blur_kernel_size=5, threshold_value=150):
    """
    Loads an image, automatically crops to the circular tank, and creates a clean binary mask of the phantom.
    
    Args:
        image_path (str): The file path to the ground truth JPG image.
        blur_kernel_size (int): The size of the kernel for Gaussian blur to aid in circle detection.
        threshold_value (int): The pixel intensity value (0-255) to separate the object from the background.

    Returns:
        numpy.ndarray: A 2D array representing the final binary mask (object is 255, background is 0).
                       Returns None if a circle cannot be detected.
    """
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.medianBlur(gray, blur_kernel_size)
    
    detected_circles = cv2.HoughCircles(gray_blurred, 
                                        cv2.HOUGH_GRADIENT, 1, 2000, 
                                        param1=50, param2=30, 
                                        minRadius=100, maxRadius=400)

    if detected_circles is not None:
        detected_circles = np.uint16(np.around(detected_circles))
        pt = detected_circles[0, 0]
        a, b, r = pt[0], pt[1], pt[2]

        circle_mask = np.zeros(gray.shape, dtype=np.uint8)
        cv2.circle(circle_mask, (a, b), r, 255, -1)
        masked_gray = cv2.bitwise_and(gray, gray, mask=circle_mask)
    else:
        print("Warning: No circle detected. Using the full image. Cropping is recommended.")
        masked_gray = gray

    _, binary_mask = cv2.threshold(masked_gray, threshold_value, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

    return binary_mask

In [ ]:
def create_ground_truth_vector(mesh_obj, binary_mask):
    """
    Maps a 2D binary mask onto a 1D pyEIT mesh element vector.

    Args:
        mesh_obj (dict): The pyEIT mesh object.
        binary_mask (numpy.ndarray): The 2D binary mask from process_image_to_mask.

    Returns:
        numpy.ndarray: A 1D vector where each element corresponds to a triangle in the mesh.
                       Value is 1 for the object, 0 for the background.
    """
    nodes = mesh_obj['node']
    elements = mesh_obj['element']
    h, w = binary_mask.shape
    
    pts = np.array(nodes)
    pts[:, 0] = (pts[:, 0] + 1) * w / 2
    pts[:, 1] = (pts[:, 1] + 1) * h / 2
    
    ground_truth_vector = np.zeros(len(elements))
    for i, element in enumerate(elements):
        p1, p2, p3 = pts[element[0]], pts[element[1]], pts[element[2]]
        centroid_x = int((p1[0] + p2[0] + p3[0]) / 3)
        centroid_y = int((p1[1] + p2[1] + p3[1]) / 3)
        
        if 0 <= centroid_y < h and 0 <= centroid_x < w:
            if binary_mask[centroid_y, centroid_x] == 255:
                ground_truth_vector[i] = 1.0

    return ground_truth_vector